# Ice Cap Sizes

This notebook reads the exploded data files and calculates the sizes of the ice caps and ice catchments in them and then adds those areas to the dataframe. Finally, the 10 largest ice caps in each region are saved to a shapefile.

These are the regions where ice caps are being evaluated:

* Region 3 - Arctic Canada, North
* Region 4 - Arctic Canada, South
* Region 5 - Greenland
* Region 6 - Iceland
* Region 7 - Svalbard and Jan Mayen (Note these are analyzed separately since they are far apart)
* Region 8 - Scandinavia
* Region 9 - Russian Arctic
* Region 10 - Asia, North
* Region 17 - Southern Andes

Adding region 16 (low latitudes) as a region to do ice cap analysis because South America has some small ice caps, but this was not initially identified as a region that needed ice cap analysis.

Adding Region 19 sub antarctic islands

In [1]:
import os
import os.path as op
import sys
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd

# set working dir
HOME = op.join(op.expanduser("~"))
os.chdir(os.path.join(HOME, "git/wgms-glacier-project"))

# Set up path to load scripts
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import scripts.wgms_scripts as ws

In [2]:
# Set up data dictionary with CRS codes for each region
crs_codes = {
            '3' : 'epsg:3347', # another possibility - 'esri:102001'
            '4' : 'epsg:3347',
            '5' : 'epsg:3178', # another possibility - egsg:32627, epsg:3995
            '6' : 'epsg:3057',
            '7' : 'epsg:32635', # region 7 svalbard only since svalbard and jan mayan are far apart. Other code - egsp:3049
            '7_jan_mayan' : 'epsg:3058', # region 7 jan mayan only since svalbard and jan mayan are far apart
            '8' : 'epsg:3049',
            '9' : 'epsg:5940', # this one is polar sterographic - should I use it????
            '10' : 'esri:102025', # another possibility - esri:102026
            '16' : 'esri:102033', # South_America_Albers_Equal_Area_Conic, since the ice caps I want to measure are in South America, going with that CRS. See https://gis.stackexchange.com/questions/111515/projected-coordinate-system-for-south-america
            '17' : 'esri:102033', # another possibility - esri:102032
            '19' : 'ESRI:102020'  # 'epsg:3031'
            }

# Set flag for Region 19 - RGI or GLIMS. Have to toggle this to do each part of region 19
r19 = 'GLIMS'

In [3]:
for region in crs_codes:
    # Set up output file name to check if it already exists. If it does, nothing to process
    # Adding the epsg or esri code to the output filename so that it is obvious which was used when calculating the area
    if region == '19':
        if r19 == 'RGI':
            output_fp = "data/rgi/processed/ice-caps/largest/largest-ice-caps-region_" + \
                str(region) + "_" + crs_codes[str(region)].replace(':', '') + ".shp"
        if r19 == 'GLIMS':
            output_fp = "data/glims/processed/ice-caps/largest/largest-ice-caps-region_" + \
                str(region) + "_" + crs_codes[str(region)].replace(':', '') + ".shp"
    else:
        output_fp = "data/glims/processed/ice-caps/largest/largest-ice-caps-region_" + \
            str(region) + "_" + crs_codes[str(region)].replace(':', '') + ".shp"
    
    if os.path.exists(output_fp) == False:
        # Open exploded region file
        print("Region: ", region)
        if region == '19':
            if r19 == 'RGI':
                region_fn = "data/rgi/processed/ice-caps/exploded/exploded_" + str(region) + ".shp"
            if r19 == 'GLIMS':
                region_fn = "data/glims/processed/ice-caps/exploded/exploded_huber_" + str(region) + ".shp"
        else:    
            region_fn = "data/glims/processed/ice-caps/exploded/exploded_" + str(region) + ".shp"
        glims_region_df = gpd.read_file(region_fn)
    
        # Determine the area of all the polygons
        region_polygon_areas = glims_region_df['geometry'].to_crs({'init': crs_codes[str(region)]}).area/10**6
    
        # Add the areas to the dataframe
        glims_region_df = glims_region_df.assign(area=region_polygon_areas)
    
        # Determine the 10 largest ice caps
        ten_largest_df = glims_region_df[['id', 'area', 'geometry']].nlargest(10, 'area')
    
        # Print 10 largest and their size in km^2
        print(ten_largest_df)
        print("")
    
        # Save ten largest dataframe for this region to shapefile
        ten_largest_df.to_file(driver='ESRI Shapefile', filename=output_fp)
    else:
        print("Region " + str(region) + " " + crs_codes[str(region)] + " file has already been processed.")
        print("")
        print("")

Region 3 epsg:3347 file has already been processed.


Region 4 epsg:3347 file has already been processed.


Region 5 epsg:3178 file has already been processed.


Region 6 epsg:3057 file has already been processed.


Region 7 epsg:32635 file has already been processed.


Region 7_jan_mayan epsg:3058 file has already been processed.


Region 8 epsg:3049 file has already been processed.


Region 9 epsg:5940 file has already been processed.


Region 10 esri:102025 file has already been processed.


Region 16 esri:102033 file has already been processed.


Region 17 esri:102033 file has already been processed.


Region:  19
    id          area                                           geometry
95  95  80852.069155  POLYGON ((-65.016774 -67.351969, -65.027069 -6...
18  18   3864.946465  POLYGON ((-68.57106 -67.734026, -68.572852 -67...
93  93   2063.249414  POLYGON ((-63.555302 -64.754957, -63.555196 -6...
79  79   1890.558057  POLYGON ((-57.920545 -64.434573, -57.920224 -6...
56  56   1447.

# Test area

In [4]:
#'''region=17
#region_fn = "data/glims/processed/ice-caps/exploded/exploded_" + str(region) + ".shp"
#glims_region_df = gpd.read_file(region_fn)
    
## Determine the area of all the polygons
#region_polygon_areas = glims_region_df['geometry'].to_crs({'init': crs_codes[str(region)]}).area/10**6
    
# Add the areas to the dataframe
#glims_region_df = glims_region_df.assign(area=region_polygon_areas)
    
# Determine the 10 largest ice caps
#ten_largest_df = glims_region_df[['id', 'area', 'geometry']].nlargest(10, 'area')
    
# Print 10 largest and their size in km^2
#print(ten_largest_df)

# Save regional dataframe to shapefile
#fp = "data/glims/processed/ice-caps/largest/largest-ice-caps-region_" + str(region) + ".shp"
#ten_largest_df.to_file(driver='ESRI Shapefile', filename=fp)'''